In [1]:
import pandas as pd
import pandas as pd
Feld_neg_AA=pd.read_csv('Feld_neg.csv').iloc[:,1:]
Feld_neg_cheflag=pd.read_csv('Feld_Fv_neg_cheflag.csv').iloc[:,1:]
NN=pd.read_csv('Neg_Feld_Y.csv').iloc[:,1:]


Feld_pos_AA=pd.read_csv('Feld_pos.csv').iloc[:,1:]
Feld_pos_cheflag=pd.read_csv('Feld_Fv_pos_cheflag.csv').iloc[:,1:]
PP=pd.read_csv('Pos_Feld_Y.csv').iloc[:,1:]



cli_train_AA=pd.read_csv('Cli_train.csv').iloc[:,1:]

cli_train_cheflag=pd.read_csv('Clinical_train_cheflag.csv').iloc[:,1:]
Train=pd.read_csv('cli_train_Y.csv').iloc[:,1:]


cli_test_AA=pd.read_csv('Cli_test.csv').iloc[:,1:]
cli_test_cheflag=pd.read_csv('Clinical_test_cheflag.csv').iloc[:,1:]
Test=pd.read_csv('cli_test_Y.csv').iloc[:,1:]




In [2]:
CColumn=['NDE_Fv',
'QDE_Fv',
'NQDE_Fv',
'IW_VH',
'RK_Fv',
'W_VH',
'D_Fv',
'W_Fv',
'G_Fv',
'G_VH',
'WRK_Fv',
'WRK_VH',
'IWR_Fv',
'WYRK_Fv',
'WYRK_VH',
'PYRK_Fv',
'IWRK_VH',
'PWRK_VH',
'PWRK_Fv',
'WRHK_Fv',
'IWRK_Fv',
'IYRK_Fv',
'FWRK_Fv',
'IPWR_Fv','Fv_charge','VH_charge']

In [3]:
Feld_neg_AA=Feld_neg_AA[CColumn]
Feld_pos_AA=Feld_pos_AA[CColumn]
cli_train_AA=cli_train_AA[CColumn]
cli_test_AA=cli_test_AA[CColumn]

In [4]:
YYColumn=['FIPWY_Fv','IPWY_VH','IWY_VH','IPWY_Fv','IPVWY_VH']

In [5]:
Feld_neg_Y=NN[YYColumn]
Feld_pos_Y=PP[YYColumn]
cli_train_Y=Train[YYColumn]
cli_test_Y=Test[YYColumn]

In [6]:
import random
Feld_neg=pd.concat([Feld_neg_AA,Feld_neg_Y,Feld_neg_cheflag],axis=1,sort=False)
Feld_pos=pd.concat([Feld_pos_AA,Feld_pos_Y,Feld_pos_cheflag],axis=1,sort=False)

cli_train=pd.concat([cli_train_AA,cli_train_Y,cli_train_cheflag],axis=1,sort=False)
cli_test=pd.concat([cli_test_AA,cli_test_Y,cli_test_cheflag],axis=1,sort=False)

cli=pd.concat([cli_train,cli_test],axis=0,sort=False).reset_index().iloc[:,1:]
Feld_all=pd.concat([Feld_neg,Feld_pos],axis=0,sort=False).reset_index().iloc[:,1:]
Type=[0]*len(Feld_neg)+[1]*len(Feld_pos)

In [7]:
Binding_train=pd.read_excel('Binding_data.xlsx',sheet_name='clinical_train')
Binding_test=pd.read_excel('Binding_data.xlsx',sheet_name='clinical_test')
Binding_PSR=pd.concat([Binding_train.iloc[:,-1],Binding_test.iloc[:,-4]],axis=0,sort=False).reset_index().iloc[:,1:]

In [8]:
features=pd.concat([Feld_all,cli],axis=0,sort=False).reset_index().iloc[:,1:].fillna(0)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
features_norm = min_max_scaler.fit_transform(features)

fst_BI1=pd.DataFrame(features_norm).iloc[:-(123),:]
fst_cli1=pd.DataFrame(features_norm).iloc[-(123):,:].reset_index().iloc[:,1:]

fst_BI1.columns=features.columns
fst_cli1.columns=features.columns

In [10]:
# Import necessary modules
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression


X_train1, X_test1, y_train1, y_test1 = train_test_split(fst_BI1, Type,test_size=0.2,random_state=42,stratify=Type)


In [11]:
## check the intermidiate RF model performance


# Import necessary modules
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

n_estimator=[5]
max_depth=[20]
min_samples_split=[20]
para=[]
result_perf=[]
result_cross=[]
bootstrap=[True,False]
for i in range(0,len(n_estimator)):
    print(i)
    for j in range(0,len(max_depth)):
        for z in range(0,len(min_samples_split)):
            for ii in range(0,1):
                regc = RandomForestClassifier(n_estimators=n_estimator[i],max_depth=max_depth[j], min_samples_split=min_samples_split[z],min_samples_leaf=1,bootstrap=bootstrap[ii],random_state=0)

                regc.fit(X_train1, y_train1)

                scores = cross_val_score(regc, X_train1, y_train1, cv=10,scoring='f1_macro')
                cross_vali_mean=scores.mean()
                cross_vali_all=scores

                y_pred_test=regc.predict(X_test1)
                y_pred_train=regc.predict(X_train1)

                y_pred_all_clinical=regc.predict(fst_cli1)

                rf_test=balanced_accuracy_score(y_test1, y_pred_test)
                rf_train=accuracy_score(y_train1, y_pred_train)
                logit_roc_auc = roc_auc_score(y_pred_all_clinical,Binding_PSR)
                print(logit_roc_auc)
                print(rf_train)
                para.append([n_estimator[i],max_depth[j],min_samples_split[z]])
                result_perf.append([rf_test,rf_train,logit_roc_auc])
                result_cross.append([cross_vali_mean,cross_vali_all[0],cross_vali_all[1],cross_vali_all[2],cross_vali_all[3],cross_vali_all[4],cross_vali_all[5],cross_vali_all[6],cross_vali_all[7],cross_vali_all[8],cross_vali_all[9]])


0
0.7510834236186349
0.9765877970299542


In [12]:
df_para=pd.DataFrame(para)
df_perf=pd.DataFrame(result_perf)
df_perf.columns=['acc_test','acc_train','auc_cli']
df_cross=pd.DataFrame(result_cross)
df=pd.concat([df_para,df_perf,df_cross],axis=1,sort=False)

In [13]:
df[df.iloc[:,6]>0.9].sort_values('auc_cli',ascending=False)

,0,1,2,acc_test,acc_train,auc_cli,0,1,2,3,4,5,6,7,8,9,10
0,5,20,20,0.952204,0.976588,0.751083,0.951127,0.953217,0.954126,0.949757,0.951684,0.953974,0.948308,0.94877,0.94847,0.951734,0.951235


In [14]:
# random forest for feature importance on a classification problem
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot
# define the model
model = RandomForestClassifier(n_estimators=5,max_depth=20, min_samples_split=20,min_samples_leaf=1,random_state=0)
# fit the model
model.fit(X_train1, y_train1)
# get importance
importance = model.feature_importances_

In [15]:
df_feature=pd.DataFrame({'feature':list(X_train1.columns),'import':list(importance)})
feature_top10=list(df_feature.sort_values(by='import',axis=0,ascending=False).head(8)['feature'])
feature_cheflag=['X1','X2','X3','X4','X5','X6','X7','X8','X9','X11','X12']
feature_top10=['Fv_charge','WYRK_VH','VH_charge','IYRK_Fv','FWRK_Fv','NDE_Fv','IPWY_VH']
fst_BI2=fst_BI1[feature_top10+feature_cheflag]
fst_cli2=fst_cli1[feature_top10+feature_cheflag]

In [16]:
# Import necessary modules
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

# Load data and split in train and test sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(fst_BI2, Type,test_size=0.2,random_state=42,stratify=Type)

In [17]:
regc = RandomForestClassifier(n_estimators=15,max_depth=15,min_samples_split=100,min_samples_leaf=1,bootstrap=True,random_state=0)

regc.fit(X_train2, y_train2)


y_pred_test=regc.predict(X_test2)
y_pred_train=regc.predict(X_train2)
y_pred_all_clinical=regc.predict(fst_cli2)
rf_test=balanced_accuracy_score(y_test2, y_pred_test)
rf_train=balanced_accuracy_score(y_train2, y_pred_train)
logit_roc_auc = roc_auc_score(y_pred_all_clinical,Binding_PSR)
                
logit_roc_auc1 = roc_auc_score(y_pred_all_clinical[:103],Binding_PSR[:103])
logit_roc_auc2 = roc_auc_score(y_pred_all_clinical[103:],Binding_PSR[103:])
print([logit_roc_auc,logit_roc_auc1,logit_roc_auc2,rf_test,rf_train])
result_perf=[]
result_perf.append([logit_roc_auc,logit_roc_auc1,logit_roc_auc2,rf_train,rf_test])

[0.8193391115926326, 0.7978682170542636, 0.9595959595959596, 0.9486752580960026, 0.9580084793176753]


In [18]:
df_perf=pd.DataFrame(result_perf)
df_perf.columns=['auc_cli_all','auc_cli_train','auc_cli_test','acc_train','acc_test',]
df_perf

,auc_cli_all,auc_cli_train,auc_cli_test,acc_train,acc_test
0,0.819339,0.797868,0.959596,0.958008,0.948675


In [20]:
#Input your sequence features and SASA here
sequence_features = pd.read_csv('Seq_features.csv', index_col = 0)
sasa_features = pd.read_csv('SASA.csv', index_col = 0)
Input = seq.merge(sasa, on = 'PreCADName', how = 'inner')
Input

,NDE_VH,QDE_VH,NQDE_VH,IW_VH,RK_VH,W_VH,D_VH,G_VH,WRK_VH,IWR_VH,...,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
PreCADName,,,,,,,,,,,,,,,,,,,,,
abituzumab,12,17,19,7,9,4,4,12,13,12,...,24.575869,20.327428,7.036527,5.400102,5.010957,3.536961,3.606572,4.954816,5.043559,5.205131
atezolizumab,11,14,16,9,9,6,5,15,15,15,...,25.444944,19.640125,5.973713,6.614986,5.983806,3.381899,4.764885,2.941195,3.728312,3.618775
brentuximab,11,16,20,10,9,5,3,12,14,10,...,27.856052,15.741025,7.413689,3.791847,4.460951,2.898559,3.149553,6.658983,4.408491,2.924984
brodalumab,9,16,18,6,12,4,4,11,16,13,...,24.918142,17.765954,4.367762,6.778272,4.270316,3.459810,4.018890,4.915614,2.464988,4.345900
carlumab,11,18,19,9,9,3,4,15,12,13,...,27.660883,17.609140,7.372773,5.219675,4.337025,2.707609,3.528741,4.712343,2.456947,3.922898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
brazikumab,15,17,21,8,11,5,7,13,16,16,...,24.501691,20.528922,5.206648,5.173498,4.190398,3.783453,3.536962,4.863833,4.145506,1.794086
monalizumab,13,20,21,6,11,5,8,12,16,12,...,27.129028,18.522143,8.956085,6.072049,5.005227,3.855371,3.759234,7.748169,4.698955,5.340126
teplizumab,14,17,22,6,14,3,7,12,17,15,...,25.317236,19.682377,5.058233,6.388941,4.210733,3.886621,3.733228,4.432888,3.215088,3.100613


In [23]:
scaler = preprocessing.MinMaxScaler()
Feld_feat = Feld_all[feature_top10+feature_cheflag]
scaler.fit(Feld_feat)
input_feat = scaler.transform(Input[feature_top10+feature_cheflag])
input_feat_df = pd.DataFrame(input_feat, columns = fst_BI2.columns)
results_df = pd.DataFrame(regc.predict_proba(input_feat_df), columns = ['0', '1'])

In [24]:
results_df.index = Input.index
results_df.to_csv("polyreactivity_results.csv")

In [25]:
results_df

,0,1
PreCADName,,
abituzumab,0.889073,0.110927
atezolizumab,0.752553,0.247447
brentuximab,0.860138,0.139862
brodalumab,0.175505,0.824495
carlumab,0.959309,0.040691
...,...,...
brazikumab,0.860148,0.139852
monalizumab,0.656042,0.343958
teplizumab,0.006010,0.993990
